In [1]:
%pylab inline

import os, sys, re, glob

import numpy as np
import pandas as pd

# import subprocess
# import nilearn as nlr
# import nipy

import nibabel 
from nilearn.masking import intersect_masks
# from nilearn._utils.compat import _basestring
from nipy.modalities.fmri.glm import FMRILinearModel

from nipype.caching import Memory as NipypeMemory
import nipype.interfaces.spm as spm

from sklearn.externals.joblib import Parallel, delayed, Memory

Populating the interactive namespace from numpy and matplotlib


In [2]:
options = {'protocol': 'SOCIAL', 
           'dataset_description': 'HCP SOCIAL experiment',   
           'dataset_dir': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\',  
           'subject_dirs': '*', 
           'output_dir': 'output', 
           'nruns': 2,
           'session_2_func': 'MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR.nii.gz', 
           'session_1_func': 'MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\tfMRI_SOCIAL_RL.nii.gz', 
           'func_write_voxel_sizes': [2.0, 2.0, 2.0], 
           'nslices': 72,
           'TR': 0.72, 
           'TA': 'TR * (1 - 1 / nslices)', 
           'nsubjects': 3,
           'exclude_these_subject_ids': [],
           'include_only_these_subject_ids': [],
           'n_jobs': 32, 
#           # PREPROC PARAMS
#            # smoothing
           'fwhm': 4.0, 
           'smooth_software': 'spm',
#           # OTHER COMMON
           'caching': True, 
           'slicer': 'ortho',  
           'cut_coords': None, 
           'threshold': 3, 
           'cluster_th': 15,   # minimum number of voxels in reported clusters ?
           'p_value': 0.05,
           't_value': 8,
           'voi_radii': 6,
           'locmax_search_radii': 20,
#           # MATLAB AND SPM SPEC
           'spm_dir': 'C:\\Program Files\\MATLAB\\spm12', 
           'matlab_exec': 'C:\\Program Files\\MATLAB\\R2017a'
          }


# hpf = 128

In [3]:
from utils.io_utils import expand_path, get_relative_path, get_abspath_relative_to_file

In [4]:
# check dataset_dir
dataset_dir = options['dataset_dir']
if not os.path.exists(dataset_dir):
    raise OSError("dataset_dir '%s' doesn't exist" % dataset_dir)
    
# check output_dir
output_dir = expand_path(options['output_dir'], relative_to=dataset_dir)
if output_dir is None:
    raise OSError(
        ("Could not expand 'output_dir' : invalid"
         " path %s (relative to directory %s)") % (options['output_dir'],
                                                   dataset_dir))

    # check subject_dirs
subject_dirs = expand_path(options['subject_dirs'], relative_to=dataset_dir)
if subject_dirs is None:
    raise OSError(
        ("Could not expand 'subject_dirs' : invalid"
         " path %s (relative to directory %s)") % (options['subject_dirs'],
                                                   dataset_dir))

In [5]:
print('\n', dataset_dir, '\n', output_dir, '\n', subject_dirs)


 C:\data\graduate_thesis\hcp_data_test_gz\ 
 C:\data\graduate_thesis\hcp_data_test_gz\output 
 C:\data\graduate_thesis\hcp_data_test_gz\*


In [6]:
# how many subjects 
nsubjects = options.get('nsubjects', np.inf)
exclude_these_subject_ids = options.get(
    'exclude_these_subject_ids', [])
include_only_these_subject_ids = options.get(
    'include_only_these_subject_ids', [])

In [7]:
[x for x in sorted(glob.glob(os.path.join(dataset_dir, subject_dirs)))
                         if os.path.isdir(x)]

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\103414_3T_tfMRI_SOCIAL_preproc',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc']

In [8]:
if isinstance(subject_dirs, str):
    subject_data_dirs = [x for x in sorted(glob.glob(os.path.join(dataset_dir, subject_dirs)))
                         if os.path.isdir(x)]
    subject_data_dirs = [os.path.join(x, re.findall(r'\d{6}', x)[0]) for x in subject_data_dirs] 
else:
    # list of subjects or subject wildcards
    subject_data_dirs = [os.path.join(dataset_dir, x)
                         for x in subject_dirs]
    aux = []
    for subject_data_dir in subject_data_dirs:
        for x in sorted(glob.glob(subject_data_dir)):
            if os.path.isdir(x):
                aux.append(x)
    subject_data_dirs = aux

    @TODO добавить обработку, если несколько участников исключаются либо, наоборот, включаются только некоторые (skip_subject = True в цикл создания объектов класса). Проверить на совпадение числа участников nsubjects к числу людей в subject_data_dirs. Получение папок участников относительно пути к функциональным файлам

In [9]:
if not subject_data_dirs:
    warnings.warn("No subject directories found for wildcard: %s" % (
        subject_dirs))

In [10]:
subject_data_dirs

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\103414_3T_tfMRI_SOCIAL_preproc\\103414',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115']

In [11]:
sess_func_wildcards = [key for key in list(options.keys())
                           if re.match("session_.+_func", key)]
# sess_onset_wildcards = [key for key in list(options.keys())
#                         if re.match("session_.+_onset", key)]
sess_ids = [re.match("session_(.+)_func", session).group(1)
                for session in sess_func_wildcards]

In [12]:
sess_func_wildcards, sess_ids

(['session_1_func', 'session_2_func'], ['1', '2'])

In [13]:
from utils.subject_data import SubjectData

In [14]:
os.path.isfile(expand_path('MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR.nii.gz', relative_to=subject_data_dirs[0]))

True

In [15]:
subjects = []

for subject_data_dir in subject_data_dirs:
    if len(subjects) == nsubjects:
        # we've had enough subjects already; end
        break
    subject_id = os.path.basename(subject_data_dir)
    subject_output_dir = os.path.join(output_dir, subject_id)    
    
    # grab functional data
    func = []
    sess_output_dirs = []
    
    for s, sess_func_wildcard in enumerate(sess_func_wildcards):
        sess_func_wildcard = options[sess_func_wildcard]  
        
        if expand_path(sess_func_wildcard, relative_to=subject_data_dir) is None:
            raise OSError(
                ("Could not expand 'sess_func_wildcard' : invalid"
                 " path %s (relative to directory %s)") % (sess_func_wildcard,
                                                           subject_data_dir))
        else: 
            sess_func = os.path.join(subject_data_dir, sess_func_wildcard)
            # sess_func = sorted(glob.glob(sess_func_wildcard))
            if not os.path.isfile(sess_func):
                raise OSError(
                ("There are no such file %s") % (sess_func))
        
        # skip session if no data found
        if not sess_func:
            warnings.warn(
                ("subject %s: No func images found for"
                 " wildcard %s; Skipping subject" % (subject_id, sess_func_wildcard)))
            continue
        
        sess_dir = os.path.dirname(sess_func)
        func.append(sess_func)
        
        # session output dir
        if os.path.basename(sess_dir) != os.path.basename(
                subject_output_dir):
            sess_output_dir = os.path.join(subject_output_dir,
                                           get_relative_path(
                                               subject_data_dir, sess_dir))
        else:
            sess_output_dir = subject_output_dir
        if not os.path.exists(sess_output_dir):
            os.makedirs(sess_output_dir)
        
        sess_output_dirs.append(sess_output_dir)
        

    # make subject data
    subject_data = SubjectData(
        subject_id=subject_id, 
        func=func,
        output_dir=subject_output_dir,
        session_output_dirs=sess_output_dirs,
        session_ids=sess_ids,
        data_dir=subject_data_dir, 
        n_sessions = len(sess_ids))

    subjects.append(subject_data)
    
if not subjects:
    warnings.warn(
        "No subjects globbed (dataset_dir=%s, subject_dir_wildcard=%s" % (
            dataset_dir, subject_dir_wildcard))


    @TODO для шаблона данных сделать через glob.glob(wildcard)
    
    Subject data
        TR=options.get('TR', None),
        drift_model='Cosine',
        hrf_model=options.get('hrf_model', 'spm + derivative'),
        hfcut=options.get("hfcut", 128.),
        time_units=options.get("time_units", "seconds")

In [16]:
subjects

[{'session_scratch_dirs': ['C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL', 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR'], 'session_ids': ['1', '2'], 'subject_id': '100307', 'anat': None, 'data_dir': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307', 'func': ['C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\tfMRI_SOCIAL_RL.nii.gz', 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR.nii.gz'], 'n_sessions': 2, 'session_output_dirs': ['C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL', 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR'], 'warpable': ['anat', 'func'], 'anat_scratc

In [17]:
len(subjects)

3

In [19]:
subjects[0].func

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\tfMRI_SOCIAL_RL.nii.gz',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR.nii.gz']

In [20]:
for subject in subjects:
    subject.mregressors_file = [os.path.join(
                                    subject.data_dir, ("MNINonLinear\\Results\\tfMRI_%s_%s\\"
                                    "Movement_Regressors.txt") % (options["protocol"], direction))
                                for direction in ['RL', 'LR']]
    print(subject.mregressors_file)
    

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\Movement_Regressors.txt', 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\100307_3T_tfMRI_SOCIAL_preproc\\100307\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\Movement_Regressors.txt']
['C:\\data\\graduate_thesis\\hcp_data_test_gz\\103414_3T_tfMRI_SOCIAL_preproc\\103414\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\Movement_Regressors.txt', 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\103414_3T_tfMRI_SOCIAL_preproc\\103414\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\Movement_Regressors.txt']
['C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\Movement_Regressors.txt', 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\Movement_Regressors.txt']


In [22]:
from utils.making_dm import read_fsl_design_file, make_paradigm_from_timing_files, make_dmtx_from_timing_files 

### проверка

In [23]:
import copy
subject_data_tmp = copy.deepcopy(subject_data)

In [24]:
subject_data_tmp.design_files = [os.path.join(
            subject_data_tmp.data_dir, ("MNINonLinear\\Results\\tfMRI_%s_%s\\"
                                    "tfMRI_%s_%s_hp200_s4_level1.fsf") % (
                options["protocol"], direction, options["protocol"], direction))
            for direction in ['RL', 'LR']]

In [25]:
subject_data_tmp.design_files

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\tfMRI_SOCIAL_RL_hp200_s4_level1.fsf',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR_hp200_s4_level1.fsf']

In [26]:
# subject_data_tmp.mregressors_file = [os.path.join(
#             subject_data_tmp.data_dir, ("MNINonLinear\\Results\\tfMRI_%s_%s\\"
#                                     "Movement_Regressors.txt") % (
#                 options["protocol"], direction))
#             for direction in ['RL', 'LR']]
# subject_data_tmp.mregressors_file

In [27]:
fsl_condition_ids, timing_files, fsl_contrast_ids, contrast_values = \
            read_fsl_design_file(subject_data_tmp.design_files[0])

In [28]:
contrast_values

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 1.,  0., -1.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [-1.,  0.,  1.,  0.]])

In [29]:
np.add([1,0,0,0],[0,0,1,0])

array([1, 0, 1, 0])

In [30]:
timing_files

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\EVs\\rnd.txt',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\EVs\\mental.txt']

In [31]:
fsl_contrast_ids

['RANDOM', 'TOM', 'RANDOM-TOM', 'neg_RANDOM', 'neg_TOM', 'TOM-RANDOM']

In [32]:
fsl_condition_ids

['RANDOM', 'TOM']

In [33]:
timing_files = [tf.replace("EVs", "tfMRI_%s_%s\\EVs" % (
                    "SOCIAL", "LR")) for tf in timing_files]

In [34]:
timing_files

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\EVs\\rnd.txt',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\105115_3T_tfMRI_SOCIAL_preproc\\105115\\MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\EVs\\mental.txt']

In [35]:
paradigm = make_paradigm_from_timing_files(timing_files)
paradigm

,duration,modulation,name,onset
0,23.0,1.0,rnd,84.031
1,23.0,1.0,rnd,160.080
2,23.0,1.0,mental,8.210
3,23.0,1.0,mental,46.007
4,23.0,1.0,mental,122.056


In [36]:
n = nibabel.load(subject_data_tmp.func[0]).shape[-1]
design_matrix, paradigm, frametimes = make_dmtx_from_timing_files(
            timing_files = timing_files, condition_ids=fsl_condition_ids, n_scans = int(n),
    tr=0.72, add_regs_file = subject_data_tmp.mregressors_file[0])

In [37]:
design_matrix.shape

(274, 17)

In [38]:
design_matrix

,RANDOM,TOM,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,drift_1,drift_2,constant
0.00,0.000000,0.000000e+00,-0.015843,0.125530,0.050378,0.122957,-0.046696,0.060619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085434,0.085430,1.0
0.72,0.000000,0.000000e+00,0.065150,0.110418,0.104669,0.075917,0.000000,0.064630,0.080993,-0.015112,0.054291,-0.047040,0.046696,0.004011,0.085423,0.085385,1.0
1.44,0.000000,0.000000e+00,-0.095092,0.114740,0.013754,0.115623,-0.029335,0.026986,-0.160242,0.004322,-0.090915,0.039706,-0.029335,-0.037644,0.085401,0.085295,1.0
2.16,0.000000,0.000000e+00,-0.052447,0.074517,-0.094087,0.127655,-0.037930,0.062281,0.042645,-0.040223,-0.107841,0.012032,-0.008595,0.035295,0.085367,0.085161,1.0
2.88,0.000000,0.000000e+00,0.086434,0.088195,-0.078090,0.095283,-0.080443,0.097002,0.138881,0.013678,0.015997,-0.032372,-0.042513,0.034721,0.085322,0.084981,1.0
3.60,0.000000,0.000000e+00,0.073575,0.117748,-0.011579,0.115852,-0.000401,0.061421,-0.012859,0.029553,0.066511,0.020569,0.080042,-0.035581,0.085266,0.084757,1.0
4.32,0.000000,0.000000e+00,0.012426,0.128448,-0.012632,0.125707,-0.028762,0.128514,-0.061149,0.010700,-0.001053,0.009855,-0.028361,0.067093,0.085199,0.084488,1.0
5.04,0.000000,0.000000e+00,0.012370,0.134122,-0.012872,0.128572,-0.035638,0.129546,-0.000056,0.005674,-0.000240,0.002865,-0.006876,0.001032,0.085120,0.084175,1.0
5.76,0.000000,0.000000e+00,0.012945,0.143390,0.015079,0.132926,-0.037873,0.134702,0.000575,0.009268,0.027951,0.004354,-0.002235,0.005156,0.085030,0.083818,1.0
6.48,0.000000,0.000000e+00,-0.045032,0.144819,-0.011652,0.115909,-0.055176,0.119748,-0.057977,0.001429,-0.026731,-0.017017,-0.017303,-0.014954,0.084929,0.083416,1.0


In [71]:
from nipy.modalities.fmri.glm import FMRILinearModel

def run_subject_level1_glm(subject_data,
                          tr=.72,
                          hrf_model="spm + derivative",
                          drift_model="Cosine",
                          regress_motion=True,
                          slicer='ortho',
                          cut_coords=None,
                          threshold=3.,
                          cluster_th=15,
                          normalize=True,
                          fwhm=0.,
                          protocol="SOCIAL",
                          func_write_voxel_sizes=None,
                          anat_write_voxel_sizes=None,
                          **other_preproc_kwargs
                          ):
    """
    Function to do preproc + analysis for a single HCP subject (task fMRI)

    """
#     @TODO переделать
#     subject_data.n_sessions = 2
    
    subject_data.tmp_output_dir = os.path.join(subject_data.output_dir, "tmp")
    if not os.path.exists(subject_data.tmp_output_dir):
        os.makedirs(subject_data.tmp_output_dir)
    if not os.path.exists(subject_data.output_dir):
        os.makedirs(subject_data.output_dir)

    cache_dir = os.path.join(subject_data.output_dir,
                                         'cache_dir')
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    mem = Memory(cache_dir)
    
    # glob design files (.fsf)
    subject_data.design_files = [os.path.join(
            subject_data.data_dir, ("MNINonLinear\\Results\\tfMRI_%s_%s\\"
                                    "tfMRI_%s_%s_hp200_s4_level1.fsf") % (
                protocol, direction, protocol, direction))
            for direction in ['RL', 'LR']]
    
    assert len(subject_data.design_files) == 2
    
    for df in subject_data.design_files:
        if not os.path.isfile(df):
            return
    
    # merged lists
    paradigms = []
    frametimes_list = []
    design_matrices = []
    n_scans = []
    
    
    # for direction, direction_index in zip(['LR', 'RL'], xrange(2)):
    for sess in range(subject_data.n_sessions):
        direction = ['RL', 'LR'][sess]
        design_file = subject_data.design_files[sess]
        if not os.path.isfile(design_file):
            print("Can't find design file %s; skipping subject %s" %
                  design_file, subject_data.subject_id)
            return
        # read the experimental setup
        print("Reading experimental setup from %s ..." % design_file)
        fsl_condition_ids, timing_files, fsl_contrast_ids, contrast_values = \
            read_fsl_design_file(design_file)
        print("... done.\r\n")
        # fix timing filenames
        timing_files = [tf.replace("EVs", "tfMRI_%s_%s\\EVs" % (
                    protocol, direction)) for tf in timing_files]
        # make design matrix
        print("Constructing design matrix for direction %s ..." % direction)
        _n_scans = nibabel.load(subject_data.func[sess]).shape[-1]
        n_scans.append(_n_scans)
#         add_regs_file = add_regs_files[
#             sess] if not add_regs_files is None else None
        add_regs_file = subject_data.mregressors_file[
            sess] 
        design_matrix, paradigm, frametimes = make_dmtx_from_timing_files(
            timing_files, fsl_condition_ids, n_scans=_n_scans, tr=tr,
            hrf_model=hrf_model, drift_model=drift_model,
            add_regs_file=add_regs_file,
            add_reg_names= None
            )

        print("... done.")
        paradigms.append(paradigm)
        frametimes_list.append(frametimes)
        design_matrices.append(design_matrix)
        
        # convert contrasts to dict
        contrasts = dict((contrast_id,
                          # append zeros to end of contrast to match design
                          np.hstack((contrast_value, np.zeros(len(
                                design_matrix.columns) - len(contrast_value))))) for contrast_id, contrast_value in zip(
                                                                            fsl_contrast_ids, contrast_values))
        contrasts['All motion'] = contrasts['RANDOM'] + contrasts['TOM']
        
        contrasts = dict((k, v) for k, v in contrasts.items() if "TOM-" in k or "All" in k)
        
        
    # replicate contrasts across sessions
    contrasts = dict((cid, [cval] * 2)
                     for cid, cval in contrasts.items())
    
    nipype_mem = NipypeMemory(base_dir=cache_dir)

    if np.sum(fwhm) > 0.:
        print('Smoothing ...')
#         subject_data.func = nipype_mem.cache(spm.Smooth)(
#             in_files=subject_data.func,
#             fwhm=fwhm,
# #             ignore_exception=False,
#             ).outputs.smoothed_files
        
#         smooth = spm.Smooth()
#         smooth.inputs.in_files = subject_data.func
#         smooth.inputs.fwhm = [4, 4, 4]
#         smooth.run() 
#         subject_data.func = smooth.outputs.smoothed_files
        print("... done.")
    
    # fit GLM
    def tortoise(*args):
        print(
            'Fitting a "Fixed Effect" GLM for merging RL and LR '
            'phase-encoding directions for subject %s ...' %
            subject_data.subject_id)
        fmri_glm = FMRILinearModel(subject_data.func,
                                   [design_matrix
                                    for design_matrix in design_matrices],
                                   mask='compute'
                                   )
        fmri_glm.fit(do_scaling=True, model='ar1')
        print(fmri_glm)
        print("... done.\r\n")

        # save computed mask
        mask_path = os.path.join(subject_data.output_dir, "mask.nii")
        print("Saving mask image to %s ..." % mask_path)
        nibabel.save(fmri_glm.mask, mask_path)
        print("... done.\r\n")

        z_maps = {}
        effects_maps = {}
        map_dirs = {}
        try:
            for contrast_id, contrast_val in contrasts.items():
                print("\tcontrast id: %s" % contrast_id)
                z_map, eff_map = fmri_glm.contrast(
                    contrast_val,
                    con_id=contrast_id,
                    output_z=True,
                    output_effects=True
                    )

                # store stat maps to disk
                for map_type, out_map in zip(['z', 'effects'],
                                             [z_map, eff_map]):
                    map_dir = os.path.join(
                        subject_data.output_dir, '%s_maps' % map_type)
                    map_dirs[map_type] = map_dir
                    if not os.path.exists(map_dir):
                        os.makedirs(map_dir)
                    map_path = os.path.join(
                        map_dir, '%s_%s.nii' % (map_type, contrast_id))
                    print("\t\tWriting %s ..." % map_path)

                    nibabel.save(out_map, map_path)

                    # collect zmaps for contrasts we're interested in
                    if map_type == 'z':
                        z_maps[contrast_id] = map_path

                    if map_type == 'effects':
                        effects_maps[contrast_id] = map_path

            return effects_maps, z_maps, mask_path, map_dirs
        except:
            return None
    
    # compute native-space maps and mask
#     stuff = mem.cache(tortoise)(
#         subject_data.func, subject_data.anat)
    stuff = tortoise(subject_data.func
                     #, subject_data.anat
                    )
    if stuff is None:
        return None
    effects_maps, z_maps, mask_path, map_dirs = stuff

    # remove repeated contrasts
    contrasts = dict((cid, cval[0]) for cid, cval in contrasts.items())
    import json
    json.dump(dict((k, list(v)) for k, v in contrasts.items()),
              open(os.path.join(subject_data.tmp_output_dir,
                                "contrasts.json"), "w"))
    subject_data.contrasts = contrasts

    subject_data.mask = mask_path
    subject_data.effects_maps = effects_maps
    subject_data.z_maps = z_maps

    return subject_data

    

In [64]:
options

{'TA': 'TR * (1 - 1 / nslices)',
 'TR': 0.72,
 'caching': True,
 'cluster_th': 15,
 'cut_coords': None,
 'dataset_description': 'HCP SOCIAL experiment',
 'dataset_dir': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\',
 'exclude_these_subject_ids': [],
 'func_write_voxel_sizes': [2.0, 2.0, 2.0],
 'fwhm': 4.0,
 'include_only_these_subject_ids': [],
 'locmax_search_radii': 20,
 'matlab_exec': 'C:\\Program Files\\MATLAB\\R2017a',
 'n_jobs': 32,
 'nruns': 2,
 'nslices': 72,
 'nsubjects': 3,
 'output_dir': 'output',
 'p_value': 0.05,
 'protocol': 'SOCIAL',
 'session_1_func': 'MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\tfMRI_SOCIAL_RL.nii.gz',
 'session_2_func': 'MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR.nii.gz',
 'slicer': 'ortho',
 'smooth_software': 'spm',
 'spm_dir': 'C:\\Program Files\\MATLAB\\spm12',
 'subject_dirs': '*',
 't_value': 8,
 'threshold': 3,
 'voi_radii': 6}

In [65]:
kwargs = {"regress_motion": True, 
          "hrf_model": "spm + derivative",
          "tr": options["TR"]
         }
kwargs.update(options)
kwargs


{'TA': 'TR * (1 - 1 / nslices)',
 'TR': 0.72,
 'caching': True,
 'cluster_th': 15,
 'cut_coords': None,
 'dataset_description': 'HCP SOCIAL experiment',
 'dataset_dir': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\',
 'exclude_these_subject_ids': [],
 'func_write_voxel_sizes': [2.0, 2.0, 2.0],
 'fwhm': 4.0,
 'hrf_model': 'spm + derivative',
 'include_only_these_subject_ids': [],
 'locmax_search_radii': 20,
 'matlab_exec': 'C:\\Program Files\\MATLAB\\R2017a',
 'n_jobs': 32,
 'nruns': 2,
 'nslices': 72,
 'nsubjects': 3,
 'output_dir': 'output',
 'p_value': 0.05,
 'protocol': 'SOCIAL',
 'regress_motion': True,
 'session_1_func': 'MNINonLinear\\Results\\tfMRI_SOCIAL_RL\\tfMRI_SOCIAL_RL.nii.gz',
 'session_2_func': 'MNINonLinear\\Results\\tfMRI_SOCIAL_LR\\tfMRI_SOCIAL_LR.nii.gz',
 'slicer': 'ortho',
 'smooth_software': 'spm',
 'spm_dir': 'C:\\Program Files\\MATLAB\\spm12',
 'subject_dirs': '*',
 't_value': 8,
 'threshold': 3,
 'tr': 0.72,
 'voi_radii': 6}

In [72]:

# n_jobs = int(os.environ.get('N_JOBS', 1))
# if n_jobs > 1:
#     subjects = Parallel(
#         n_jobs=n_jobs, verbose=100)(delayed(
#             run_suject_level1_glm)(
#                 subject_data,
#                 **kwargs) for subject_data in subjects)
# else:


subjects_glm = [run_subject_level1_glm(subject_data,
                                      **kwargs)
                for subject_data in subjects]
subjects_glm = [subject for subject in subjects if subject]



Reading experimental setup from C:\data\graduate_thesis\hcp_data_test_gz\100307_3T_tfMRI_SOCIAL_preproc\100307\MNINonLinear\Results\tfMRI_SOCIAL_RL\tfMRI_SOCIAL_RL_hp200_s4_level1.fsf ...
... done.

Constructing design matrix for direction RL ...
... done.
Reading experimental setup from C:\data\graduate_thesis\hcp_data_test_gz\100307_3T_tfMRI_SOCIAL_preproc\100307\MNINonLinear\Results\tfMRI_SOCIAL_LR\tfMRI_SOCIAL_LR_hp200_s4_level1.fsf ...
... done.

Constructing design matrix for direction LR ...
... done.
Smoothing ...
... done.
Fitting a "Fixed Effect" GLM for merging RL and LR phase-encoding directions for subject 100307 ...
... done.

Saving mask image to C:\data\graduate_thesis\hcp_data_test_gz\output\100307\mask.nii ...
... done.

	contrast id: TOM-RANDOM
		Writing C:\data\graduate_thesis\hcp_data_test_gz\output\100307\z_maps\z_TOM-RANDOM.nii ...
		Writing C:\data\graduate_thesis\hcp_data_test_gz\output\100307\effects_maps\effects_TOM-RANDOM.nii ...
	contrast id: All motion
		W

In [73]:
mask_images = [subject_data.mask for subject_data in subjects]
mask_images

['C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\mask.nii',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\103414\\mask.nii',
 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\105115\\mask.nii']

In [74]:
nibabel.load(mask_images[0]).shape

(91, 109, 91)

In [136]:
# level 2
from pypreprocess.reporting.glm_reporter import (generate_subject_stats_report,
                                                 group_one_sample_t_test)



In [80]:
task_output_dir = os.path.join(os.path.dirname(subject_data_tmp.output_dir))

In [85]:
for mi in mask_images:
    print(nibabel.load(mi).shape)

(91, 109, 91)
(91, 109, 91)
(91, 109, 91)


In [87]:
nibabel.load(mask_images[0]).affine

array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [93]:
intersected_mask = intersect_masks(mask_images)

In [95]:
intersected_mask.shape

(91, 109, 91)

In [96]:
af = nibabel.load(mask_images[0]).affine
af.shape

array([[  -2.,    0.,    0.,   90.],
       [   0.,    2.,    0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [99]:
group_mask = nibabel.Nifti1Image(
    intersected_mask, #.astype(np.int8),
    affine = af)

In [102]:
nibabel.save(group_mask, os.path.join(
        task_output_dir, "mask.nii.gz"))
group_mask.to_filename(os.path.join(
        task_output_dir, "mask.nii.gz")) 

In [103]:
print("... done.\r\n")
print("Group GLM")
contrasts = subjects[0].contrasts
subjects_effects_maps = [subject_data.effects_maps
                         for subject_data in subjects]

... done.

Group GLM


In [104]:
subjects_effects_maps

[{'All motion': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\effects_maps\\effects_All motion.nii',
  'TOM-RANDOM': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\100307\\effects_maps\\effects_TOM-RANDOM.nii'},
 {'All motion': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\103414\\effects_maps\\effects_All motion.nii',
  'TOM-RANDOM': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\103414\\effects_maps\\effects_TOM-RANDOM.nii'},
 {'All motion': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\105115\\effects_maps\\effects_All motion.nii',
  'TOM-RANDOM': 'C:\\data\\graduate_thesis\\hcp_data_test_gz\\output\\105115\\effects_maps\\effects_TOM-RANDOM.nii'}]

In [105]:
# group_one_sample_t_test(
#     mask_images,
#     subjects_effects_maps,
#     contrasts,
#     task_output_dir,
#     threshold=threshold,
#     cluster_th=cluster_th,
#     subjects=[subject_data.subject_id for subject_data in subjects],
#     title='Group GLM for HCP fMRI %s protocol (%i subjects)' % (
#         protocol, len(subjects)),
#     slicer=slicer
#     )